<a href="https://colab.research.google.com/github/Ehsan-Roohi/Gas-Dynamics-/blob/main/Thin_Airfoil_Theory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title **Thin Airfoil Theory AI Lab (Clean Layout - No Overlap)**
# @markdown Please click the **Play** button. Wait ~20s for training.

import os
# Force CPU usage
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
import ipywidgets as widgets
from ipywidgets import interact

# ==========================================
# 1. Physics & Geometry Functions
# ==========================================

def get_cosine_spaced_points(n):
    theta = np.linspace(0, np.pi, n)
    x = 0.5 * (1 - np.cos(theta))
    return x

N_POINTS = 80
X_COORDS = get_cosine_spaced_points(N_POINTS)

def get_camber_line(m, p, x):
    yc = np.zeros_like(x)
    if p == 0: return yc
    p = np.clip(p, 1e-6, 1 - 1e-6)
    front = x <= p
    yc[front] = (m / p**2) * (2*p*x[front] - x[front]**2)
    back = x > p
    yc[back] = (m / (1-p)**2) * ((1-2*p) + 2*p*x[back] - x[back]**2)
    return yc

def get_thickness_distribution(t, x):
    x_safe = np.clip(x, 1e-8, 1.0)
    yt = 5 * t * (0.2969*np.sqrt(x_safe) - 0.1260*x_safe - 0.3516*x_safe**2 + 0.2843*x_safe**3 - 0.1015*x_safe**4)
    yt = yt - x_safe * yt[-1]
    return yt

def solve_analytical(yc, x):
    theta = np.arccos(1 - 2*x)
    dy_dx = np.gradient(yc, x)
    mask = np.isfinite(dy_dx) & (x > 1e-6) & (x < 1 - 1e-6)
    if np.sum(mask) < 10: return 0.0, 0.0
    theta_clean = theta[mask]
    dydx_clean = dy_dx[mask]

    integrand1 = dydx_clean * (1 - np.cos(theta_clean))
    a0 = -(1 / np.pi) * np.trapz(integrand1, theta_clean)

    integrand2 = dydx_clean * (np.cos(2*theta_clean) - np.cos(theta_clean))
    cm = 0.5 * np.trapz(integrand2, theta_clean)
    return a0, cm

# ==========================================
# 2. Model Training
# ==========================================
print(">>> Generating dataset and training Neural Network... (Please wait)")

X_train, Y_train = [], []
for _ in range(3000):
    m = np.random.uniform(0.0, 0.08)
    p = np.random.uniform(0.2, 0.7)
    yc = get_camber_line(m, p, X_COORDS)
    a0, cm = solve_analytical(yc, X_COORDS)
    if np.abs(a0) < 0.5:
        X_train.append(yc)
        Y_train.append([a0, cm])

model = models.Sequential([
    layers.Input(shape=(N_POINTS,)),
    layers.Dense(128, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='linear')
])

model.compile(optimizer='adam', loss='mse')
model.fit(np.array(X_train), np.array(Y_train), epochs=60, batch_size=64, verbose=0)

print(">>> Training Complete!")
print("-" * 60)

# ==========================================
# 3. Interactive Visualization
# ==========================================

def plot_interactive(Max_Camber_Percent, Camber_Position_Tenths):
    # Parameters
    m = Max_Camber_Percent / 100.0
    p = Camber_Position_Tenths / 10.0

    # Calc
    yc = get_camber_line(m, p, X_COORDS)
    yt = get_thickness_distribution(0.12, X_COORDS)
    yu = yc + yt
    yl = yc - yt

    pred = model.predict(yc.reshape(1, -1), verbose=0)[0]
    true_a0, true_cm = solve_analytical(yc, X_COORDS)

    # --- PLOTTING SETUP ---
    fig = plt.figure(figsize=(20, 14)) # Increased height slightly

    # hspace=0.6 adds MORE space between Top Row and Bottom Row to fit the legend
    gs = fig.add_gridspec(2, 2, height_ratios=[1, 1.2], hspace=0.6, wspace=0.3)

    # Fonts
    label_font = {'size': 22, 'weight': 'bold'}
    title_font = {'size': 22, 'weight': 'bold'}
    legend_prop = {'size': 22, 'weight': 'bold'}
    tick_size = 18

    # Plot A: Geometry
    ax_geo = fig.add_subplot(gs[0, :])
    ax_geo.fill_between(X_COORDS, yu, yl, color='lightgray', label='Airfoil Body')
    ax_geo.plot(X_COORDS, yu, 'k-', linewidth=3)
    ax_geo.plot(X_COORDS, yl, 'k-', linewidth=3)
    ax_geo.plot(X_COORDS, yc, 'r--', linewidth=3, label='Mean Camber Line')

    # y=1.1 pushes the title UP, away from the airfoil
    ax_geo.set_title(f"Geometry: NACA {int(Max_Camber_Percent)}{int(Camber_Position_Tenths)}12", **title_font, y=1.05)
    ax_geo.set_aspect('equal')
    ax_geo.axis('off')
    ax_geo.set_xlim(-0.05, 1.05)

    # --- FIXED LEGEND POSITION ---
    # Put Legend BELOW the plot (bbox_to_anchor y is negative)
    # This prevents it from hitting the title (which is on top)
    ax_geo.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
                  prop=legend_prop, framealpha=1.0, ncol=2, borderaxespad=0.)

    # Alphas
    alphas_deg = np.linspace(-5, 15, 50)
    alphas_rad = np.radians(alphas_deg)

    # Plot B: Lift Coefficient
    ax_cl = fig.add_subplot(gs[1, 0])
    cl_true = 2 * np.pi * (alphas_rad - true_a0)
    cl_pred = 2 * np.pi * (alphas_rad - pred[0])

    ax_cl.plot(alphas_deg, cl_true, 'k-', linewidth=5, alpha=0.4, label='Exact Theory')
    ax_cl.plot(alphas_deg, cl_pred, 'b--', linewidth=3, label='AI Prediction')

    error_a0 = abs(np.degrees(true_a0 - pred[0]))
    ax_cl.set_title(f"Lift Coefficient ($C_l$)\nError: {error_a0:.3f}°", **title_font)
    ax_cl.set_ylabel("$C_l$", **label_font)
    ax_cl.set_xlabel("Angle of Attack (deg)", **label_font)
    ax_cl.grid(True, alpha=0.5, linewidth=1.5)
    ax_cl.tick_params(axis='both', which='major', labelsize=tick_size)
    for label in ax_cl.get_xticklabels() + ax_cl.get_yticklabels():
        label.set_fontweight('bold')
    ax_cl.legend(prop=legend_prop)

    # Plot C: Moment Coefficient
    ax_cm = fig.add_subplot(gs[1, 1])
    ax_cm.axhline(true_cm, color='k', linewidth=5, alpha=0.4, label='Exact Theory')
    ax_cm.plot(alphas_deg, [pred[1]]*len(alphas_deg), 'r--', linewidth=3, label='AI Prediction')

    error_cm = abs(true_cm - pred[1])
    ax_cm.set_title(f"Moment Coefficient ($C_m$)\nError: {error_cm:.4f}", **title_font)
    ax_cm.set_ylabel("$C_m$", **label_font)
    ax_cm.set_xlabel("Angle of Attack (deg)", **label_font)
    ax_cm.set_ylim(min(true_cm, pred[1])-0.05, max(true_cm, pred[1])+0.05)
    ax_cm.grid(True, alpha=0.5, linewidth=1.5)
    ax_cm.tick_params(axis='both', which='major', labelsize=tick_size)
    for label in ax_cm.get_xticklabels() + ax_cm.get_yticklabels():
        label.set_fontweight('bold')

    plt.tight_layout()
    plt.show()

# Widgets
style = {'description_width': 'initial'}
layout = widgets.Layout(width='800px')
interact(plot_interactive,
         Max_Camber_Percent=widgets.FloatSlider(min=0, max=6, step=0.5, value=2, description='Max Camber (%)', style=style, layout=layout),
         Camber_Position_Tenths=widgets.FloatSlider(min=2, max=7, step=0.5, value=4, description='Camber Pos (/10)', style=style, layout=layout));

>>> Generating dataset and training Neural Network... (Please wait)


/tmp/ipython-input-3516719840.py:52: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  a0 = -(1 / np.pi) * np.trapz(integrand1, theta_clean)
/tmp/ipython-input-3516719840.py:55: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  cm = 0.5 * np.trapz(integrand2, theta_clean)


>>> Training Complete!
------------------------------------------------------------


interactive(children=(FloatSlider(value=2.0, description='Max Camber (%)', layout=Layout(width='800px'), max=6…